### Setting paths and imports

In [1]:
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
module_path

os.chdir(module_path)

# from src.evaluation.evaluator_manager import EvaluatorManager
# from src.evaluation.future.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.utils.context import Context
from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

from src.evaluation.future.evaluator_manager_triplets import EvaluatorManager

config_f_name = 'generate_minimize/tcr-ablation-nodes/dce/dce-lcls-128/generate_minimize0.jsonc'

### Evaluating the explainer

In [2]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)
eval_manager.evaluate()

Generating context for: d:\escuela\4to\Tesis\repo\GRETEL\lab\config\generate_minimize/bbbp/dcm/dcm-dummy/generate_minimize0.jsonc
2025-01-07 08:36:40,1090212613 | INFO | 16124 - Executing: d:\escuela\4to\Tesis\repo\GRETEL\lab\config\generate_minimize/bbbp/dcm/dcm-dummy/generate_minimize0.jsonc Run: 1
2025-01-07 08:36:40,1090212623 | INFO | 16124 - Creating the evaluation manager....................................
2025-01-07 08:36:40,1090212639 | INFO | 16124 - Creating the evaluators......................................................
2025-01-07 08:36:40,1090212884 | INFO | 16124 - Creating: BBBP-c0cc58bdf4acf93936322304bf6b669e
2025-01-07 08:36:40,1090212885 | INFO | 16124 - Instantiating: src.dataset.generators.bbbp.BBBP


[08:36:40] Explicit valence for atom # 6 N, 4, is greater than permitted
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] Explicit valence for atom # 12 N, 4, is greater than permitted
[08:36:40] Explicit valence for atom # 6 N, 4, is greater than permitted
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] Explicit valence for atom # 5 N, 4, is greater than permitted
[08:36:40] Explicit valence for atom # 5 N, 4, is greater than permitted
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] WARNING: not removing hydrogen atom without neighbors
[08:36:40] Explicit valence for atom # 5 N, 4, is greater than permitted
[08:36:40] WARNING: not removing hydrogen

2025-01-07 08:36:42,1090215049 | INFO | 16124 - Apply: src.dataset.manipulators.padding.AdjacencyMatrixPadder


[08:36:42] WARNING: not removing hydrogen atom without neighbors
[08:36:42] WARNING: not removing hydrogen atom without neighbors


2025-01-07 08:36:42,1090215050 | INFO | 16124 - Instantiating: src.dataset.manipulators.padding.AdjacencyMatrixPadder
2025-01-07 08:36:43,1090215640 | INFO | 16124 - Saved: BBBP-c0cc58bdf4acf93936322304bf6b669e
2025-01-07 08:36:43,1090215664 | INFO | 16124 - Created: BBBP-c0cc58bdf4acf93936322304bf6b669e
2025-01-07 08:36:43,1090215675 | INFO | 16124 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-01-07 08:36:43,1090215698 | INFO | 16124 - Instantiating: torch.optim.RMSprop
2025-01-07 08:36:43,1090215699 | INFO | 16124 - Instantiating: torch.nn.CrossEntropyLoss
2025-01-07 08:36:43,1090215746 | INFO | 16124 - Creating: OracleTorch-67fdf45735a1c8aaece5ab786ca3c38e
2025-01-07 08:36:44,1090216628 | INFO | 16124 - epoch = 0 ---> loss = 0.5457	 accuracy = 0.7596
2025-01-07 08:36:44,1090217165 | INFO | 16124 - epoch = 1 ---> loss = 0.4999	 accuracy = 0.7858
2025-01-07 08:36:45,1090217704 | INFO | 16124 - epoch = 2 ---> loss = 0.4886	 accuracy = 0.7863
2025-01-07 08:36:45,1090218231 | INF

In [3]:
exp_inst = []
for exp in eval_manager.evaluators[0]._explanations:
    exp.input_instance._dataset = None
    exp.counterfactual_instances[0]._dataset = None
    
exp_inst = [(exp.input_instance, exp.counterfactual_instances[0]) for exp in eval_manager.evaluators[0]._explanations]

In [4]:
import os
import pickle

pickle_path = os.path.join(module_path, 'lab', 'pickles', 'union.pkl')
pickle_dir = os.path.dirname(pickle_path)

if not os.path.exists(pickle_dir):
    os.makedirs(pickle_dir)

with open(pickle_path, 'wb') as pickle_file:
    pickle.dump(exp_inst, pickle_file)

In [5]:
with open(os.path.join(module_path, 'lab', 'pickles', 'union.pkl'), 'rb') as clear_file:
    clear_exp = pickle.load(clear_file)

### Aggregating the stats

In [6]:
results_path = os.path.join(module_path, 'lab', 'output', 'results')
stats_file_path = os.path.join(module_path, 'lab', 'stats', 'results.csv')
res = data_analyzer.create_aggregated_dataframe(results_path)
res.to_csv(stats_file_path)
res

,scope,dataset,oracle,explainer,Runtime,Runtime-std,GraphEditDistance,GraphEditDistance-std,FeatureEditDistance,FeatureEditDistance-std,Correctness,Correctness-std,OracleCalls,OracleCalls-std,OracleAccuracy,OracleAccuracy-std,Sparsity,Sparsity-std
0,meta,ASD-15273954d84e872cf0b021cd4477bfdc,ASDOracle-9e4f3586dc330143b7849fc540b25739,GenerateMinimize(DCM-Dummy),0.003581,0.0,1029.363636,0.0,0.000000,0.0,1.0,0.0,3.0,0.0,0.909091,0.0,2.317785,0.0
1,meta,BBBP-c0cc58bdf4acf93936322304bf6b669e,OracleTorch-67fdf45735a1c8aaece5ab786ca3c38e,GenerateMinimize(DCM-Dummy),0.019772,0.0,18.578431,0.0,217.903179,0.0,1.0,0.0,3.0,0.0,0.872549,0.0,0.126793,0.0


### Visualizing individual instances

In [7]:
evaluator = eval_manager.evaluators[0]
evaluator

In [8]:
exp = evaluator.explanations[2]
og_inst = exp.input_instance
cf_inst = exp.counterfactual_instances[0]

# Drawing Instances

In [9]:
import pickle

with open(os.path.join(module_path, 'lab', 'pickles', 'cf2.pkl'), 'rb') as cf2_file:
    cf2_exp = pickle.load(cf2_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'clear.pkl'), 'rb') as clear_file:
    clear_exp = pickle.load(clear_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'irand.pkl'), 'rb') as irand_file:
    irand_exp = pickle.load(irand_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'obs.pkl'), 'rb') as obs_file:
    obs_exp = pickle.load(obs_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'rsgg.pkl'), 'rb') as rsgg_file:
    rsgg_exp = pickle.load(rsgg_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'bidirectional.pkl'), 'rb') as bi_file:
    bi_exp = pickle.load(bi_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'union.pkl'), 'rb') as union_file:
    union_exp = pickle.load(union_file)


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\escuela\\4to\\Tesis\\repo\\GRETEL\\lab\\pickles\\cf2.pkl'

In [ ]:
i = 7

og_inst = cf2_exp[i][0]
cf2_cf = cf2_exp[i][1]
print(og_inst.id, cf2_cf.id)

clear_cf = clear_exp[i][1]
print(clear_exp[i][0].id, clear_cf.id)

irand_cf = irand_exp[i][1]
print(irand_exp[i][0].id, irand_cf.id)

obs_cf = obs_exp[i][1]
print(obs_exp[i][0].id, obs_cf.id)

rsgg_cf = rsgg_exp[i][1]
print(rsgg_exp[i][0].id, rsgg_cf.id)

union_cf = union_exp[i][1]
print(union_exp[i][0].id, union_cf.id)

bi_cf = bi_exp[i][1]
print(bi_exp[i][0].id, bi_cf.id)

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf2_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, clear_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, irand_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, obs_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, rsgg_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, union_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, bi_cf, position=pos)

# End Paper Experiments

In [ ]:

changes = data_analyzer.get_cf_changes(og_inst, cf_inst, False)

In [ ]:
common_edges = changes['common edges']
added_edges = changes['added edges']
removed_edges = changes['removed edges']
common_nodes = changes['common nodes']
added_nodes = changes['added nodes']
removed_nodes = changes['removed nodes']

print(f'added edges: {added_edges}')
print(f'removed_edges: {removed_edges}')
print(f'added nodes: {added_nodes}')
print(f'removed nodes: {removed_nodes}')

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf_inst, position=pos)